In [9]:
import cv2

def preprocessing():
    image = cv2.imread("image/image2.png")

    if image is None: return None, None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imshow("gray1", gray)

    gray = cv2.equalizeHist(gray)

    cv2.imshow("gray2", gray)

    return image, gray


face_cascade = cv2.CascadeClassifier("data/haarcascade_frontalface_alt2.xml")
eys_cascade = cv2.CascadeClassifier("data/haarcascade_eye_tree_eyeglasses.xml")

image, gray = preprocessing()


if image is None: raise Exception("사진 읽기 에러")


faces = face_cascade.detectMultiScale(gray, 1.1, 2, 0, (100, 100))

if faces.any():

    x, y, w, h = faces[0]

    face_image = image[y:y+h, x:x+w]


    eyes = eys_cascade.detectMultiScale(face_image, 1.15, 7, 0, (25, 20))


    if len(eyes) == 2:
        for ex, ey, ew, eh in eyes:

            center = (x + ex + ew // 2, y + ey + eh // 2)
            cv2.circle(image, center, 10, (0, 255, 0), 2)


    else:
        print("눈 미검출")

    cv2.rectangle(image, faces[0], (255, 0, 0), 4)
    cv2.imshow("MyFace", image)

else:
    print("얼굴 미검출")

cv2.waitKey(0)


-1